## Modelo de Monte Carlo para la valuación de opciones europeas

#### _European Option Pricing with Montecarlo_



#### Ventajas y limitaciones del enfoque de Monte Carlo

#### Ventajas:

- Flexibilidad: el modelo de Monte Carlo es aplicable a una amplia variedad de opciones y activos subyacentes, incluyendo opciones personalizadas que sean más difíciles de adaptar desde Black-Scholes.


#### Limitaciones:

- Tiempo de cálculo: a medida que aumenta el número de simulaciones, el tiempo de cálculo también aumenta.
- Precisión: el valor estimado de la opción puede tener cierta variabilidad debido al carácter aleatorio de las simulaciones.

### Matemáticas necesarias

Puedes valuar una opción sin utilizar el modelo de Black-Scholes y en su lugar para simular el precio del activo subyacente en diferentes momentos, utilizar la ecuación de Movimiento Geométrico Browniano [_(Geometric Brownian Motion)_](https://en.wikipedia.org/wiki/Geometric_Brownian_motion), que se define como:

$$
S_t = S_0 \cdot e^{(r - \frac{\sigma^2}{2})t + \sigma\sqrt{t}Z}
$$


Donde:

$S_t$: Precio del activo subyacente en el tiempo $t$

$S_0$: Precio inicial del activo subyacente

$r$: Tasa de interés libre de riesgo

$\sigma$: Volatilidad del activo subyacente

$t$: Tiempo

$Z$: Variable aleatoria normal estándar (con media 0 y desviación estándar 1)

A continuación, calculamos el valor esperado del pago (valor intrínseco) de la opción en cada trayectoria simulada y descontamos el valor a la fecha presente utilizando la tasa de interés libre de riesgo.

Para una opción europea call, el valor intrínseco es:

$ C_T = max(S_T − K,0) $

Para una opción europea put, el valor intrínseco es:

$ P_T = max(K - S_T,0) $

Donde $K$ es el precio strike de la opción.

El valor esperado de la opción se calcula como la media de los pagos descontados de todas las trayectorias simuladas y se expresa como:

$$
C_0 = e^{-rT} \cdot \frac{1}{N} \sum_{i=1}^{N} C_{T,i}
$$


$$
P_0 = e^{-rT} \cdot \frac{1}{N} \sum_{i=1}^{N} P_{T,i}
$$


**Donde:**

**$C_0$ y $P_0$:** Valor presente de la opción de compra y de venta, respectivamente.

**$T$:** Tiempo hasta el vencimiento de la opción.

**$N$:** Número de trayectorias simuladas.

**$C_{T,i}$ y $P_{T,i}$:** Pago de la opción de compra y de venta en la trayectoria $i$, respectivamente.

$r$: tasa libre de riesgo

##### Pequeña explicación
Estas fórmulas representan el precio actual (también conocido como valor presente) de una cartera de opciones de compra (C) y de venta (P), respectivamente. La estructura de ambas fórmulas es muy similar, y ambas tienen en cuenta el valor presente de los flujos de efectivo futuros.


Para entender mejor cada elemento de las ecuaciones, vamos a desglosarlas:

$C_0$ y $P_0$: estos son los precios actuales de las carteras de opciones de compra y de venta, respectivamente. Se calculan a partir del valor presente de los flujos de efectivo futuros.

$e^{-rT}$: esta es la tasa de descuento, necesaria para convertir a valor presente el valor de las opciones al vencimiento. r es la tasa de interés libre de riesgo, y T es el tiempo en años que falta para el vencimiento de las opciones.

$\frac{1}{N} \sum_{i=1}^{N} C_{T,i}$ y $\frac{1}{N} \sum_{i=1}^{N} P_{T,i}$: estas son las medias aritméticas de los precios de las opciones de compra y de venta que forman la cartera. Cada $C_{T,i}$ y $P_{T,i}$ representa el precio de una opción individual en el momento T.


Ahora, implementemos esto en Python y usemos gráficas para visualizar los resultados.

**Importación de las bibliotecas necesarias**

In [43]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

**Importación de los datos necesarios**

In [44]:

ticker = 'SPY'  # Ticker de ETF SP500
data = yf.download(ticker, start='2022-01-01', end='2024-01-01')['Adj Close']

[*********************100%%**********************]  1 of 1 completed


In [45]:
data

Date
2022-01-03    463.045990
2022-01-04    462.890900
2022-01-05    454.002411
2022-01-06    453.575928
2022-01-07    451.782715
                 ...    
2023-12-22    473.649994
2023-12-26    475.649994
2023-12-27    476.510010
2023-12-28    476.690002
2023-12-29    475.309998
Name: Adj Close, Length: 501, dtype: float64

**Cálculo del rendimiento diario y la volatilidad anualizada del activo subyacente**

Estas líneas de código se utilizan para calcular la volatilidad anualizada de un activo subyacente, que es un componente clave en la valuación de opciones. Es importante tener en cuenta que el cálculo de la volatilidad implica el uso de logaritmos, que son válidos en el contexto de las finanzas.

El cálculo del rendimiento diario del activo subyacente se realiza a partir de la serie de precios diarios del activo. La función "np.log" se utiliza para calcular el logaritmo natural del cociente de los precios del activo en dos días consecutivos. Al tomar el logaritmo de este cociente, estamos calculando la variación porcentual entre los dos precios y no simplemente la diferencia de precios. Luego, se elimina cualquier valor NaN (not a number) con la función "dropna()" para asegurarnos de que los cálculos posteriores sean precisos.

Una vez que se han calculado los rendimientos diarios del activo subyacente, se utiliza la desviación estándar anualizada de estos rendimientos para calcular la volatilidad anualizada del activo subyacente. La volatilidad anualizada se calcula multiplicando la desviación estándar de los rendimientos diarios por la raíz cuadrada de 252 (que es el número de días de negociación en un año típico). El resultado es la volatilidad anualizada del activo subyacente.

El uso de logaritmos es válido en finanzas porque es común asumir que el rendimiento de los activos financieros tiende a seguir una distribución lognormal. Esto significa que los rendimientos tienden a estar más cerca de la media y tienen una cola más larga hacia los valores extremos. El uso de logaritmos ayuda a manejar los valores extremos de los rendimientos y hacer que los cálculos sean más precisos. Además, el uso de logaritmos permite sumar los rendimientos en lugar de multiplicarlos, lo que hace que los cálculos sean más fáciles.

In [46]:
daily_returns = np.log(data / data.shift(1)).dropna()  # Cálculo del rendimiento diario del activo subyacente
sigma = daily_returns.std() * np.sqrt(252)  # Cálculo de la volatilidad anualizada del activo subyacente

In [47]:
daily_returns

Date
2022-01-04   -0.000335
2022-01-05   -0.019389
2022-01-06   -0.000940
2022-01-07   -0.003961
2022-01-10   -0.001245
                ...   
2023-12-22    0.002008
2023-12-26    0.004214
2023-12-27    0.001806
2023-12-28    0.000378
2023-12-29   -0.002899
Name: Adj Close, Length: 500, dtype: float64

**Definición de los parámetros para la simulación de Monte Carlo**

In [48]:

S0 = data[-1]     # Precio inicial del activo subyacente (último precio disponible)
K = S0 * 1.1           # Precio strike de las opciones
T = 365       # Tiempo hasta el vencimiento de las opciones (en días)
r = 0.05          # Tasa de interés libre de riesgo
num_simulations = 30000  # Número de trayectorias simuladas para la valuación

In [49]:
S0

475.30999755859375

**Simulación de Monte Carlo para generar una serie de precios posibles para el activo subyacente en la fecha de vencimiento de las opciones**

Las siguientes dos líneas de código se utilizan para generar trayectorias simuladas del precio del activo subyacente.

La primera línea de código genera un conjunto de números aleatorios normalmente distribuidos utilizando la función "np.random.normal". Estos números se utilizan para simular la variación aleatoria en el precio del activo subyacente. La función "np.random.normal" toma tres argumentos: la media (que es cero en este caso), la desviación estándar (que es uno en este caso) y el número de simulaciones que se desean generar (que es el valor de "num_simulations").

La segunda línea de código utiliza los números aleatorios generados anteriormente para simular los precios del activo subyacente en diferentes momentos en el futuro. La fórmula utilizada se basa en el modelo de movimiento geométrico Browniano.La fórmula tiene en cuenta el precio actual del activo subyacente (S0), la volatilidad anualizada del activo subyacente (sigma), el tiempo hasta el vencimiento de las opciones (T), la tasa de interés libre de riesgo (r) y los números aleatorios generados anteriormente (Z).

In [50]:
S0 = data[-1]
lista = []
lista_s0 = [S0]

for i in range(365):
    
    T = 1/365
    Z = np.random.normal(0, 1, num_simulations)  # Generación de números aleatorios para la simulación de Monte Carlo
    St = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)  # Cálculo de los precios simulados para el activo subyacente
    
    lista_s0.append(St)
    S0 = lista_s0[-1]
    
    lista.append(St)

df_precios_diarios_sim = pd.DataFrame(lista)
df_precios_diarios_sim

,0,1,2,3,4,5,6,7,8,9,...,29990,29991,29992,29993,29994,29995,29996,29997,29998,29999
0,474.054555,463.891015,463.204822,475.122814,471.013489,472.228389,467.579699,468.196467,468.233326,479.078139,...,479.872499,475.804422,472.478541,469.332010,473.769460,472.114247,477.983418,474.594621,474.725363,471.915865
1,477.065648,456.488978,462.921737,475.473649,464.367649,474.406852,466.120301,471.517326,475.332194,481.693581,...,487.773972,473.333356,466.472379,457.937119,471.813619,469.974148,487.038099,476.785174,478.396085,472.494875
2,484.795551,451.353018,460.958492,478.567003,465.582742,483.506577,469.124540,470.186187,476.353288,485.226637,...,481.039294,477.215537,465.771555,454.631425,475.926968,466.878270,493.204849,476.350206,477.850141,475.454874
3,484.769324,457.764181,459.140111,472.619031,462.602443,482.435967,476.155397,467.151716,481.063416,487.661470,...,472.131475,478.131446,466.550928,451.623980,478.210631,459.500838,499.351946,485.157101,481.887622,469.236129
4,486.268795,463.908214,464.073920,472.503184,465.032108,482.151831,475.946004,470.169303,488.893143,476.817131,...,479.423359,475.242005,467.757216,454.813840,487.141631,458.917082,496.693323,482.739824,484.171658,461.704849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,422.956387,615.284190,437.511082,448.380501,561.021320,604.221395,450.038738,391.645660,488.373149,789.934888,...,441.835794,401.982101,314.254592,417.488798,444.940936,608.734682,574.066790,411.522197,452.335085,528.918124
361,435.613802,622.972687,442.391957,445.435269,555.386336,601.994376,457.699689,393.043789,487.585979,792.628744,...,437.490321,401.872524,312.852548,422.160200,443.074890,614.807899,577.538623,405.091582,444.908663,537.876566
362,433.263211,614.243579,441.988434,450.476104,553.605365,612.623133,463.545026,393.220761,490.918646,792.117667,...,437.561657,404.942388,310.433764,423.997858,442.382417,612.373475,569.128972,401.254544,438.392612,541.628650
363,429.331260,609.297179,444.290148,452.205738,554.731710,609.749172,465.565035,392.565703,483.281764,785.271730,...,442.423708,399.203547,312.742538,420.960904,436.953789,610.112453,577.177591,406.809731,442.815837,541.173345


In [51]:
media_trayectorias = df_precios_diarios_sim.mean()
media_trayectorias

0        472.816968
1        522.590071
2        444.770279
3        456.111064
4        504.269298
            ...    
29995    486.881592
29996    531.808122
29997    431.199031
29998    486.487131
29999    438.553897
Length: 30000, dtype: float64

**Cálculo del pago de las opciones para cada una de las trayectorias simuladas**

Estas líneas de código se utilizan para calcular el pago de una opción de compra (call) y una opción de venta (put) respectivamente, a partir de los precios simulados del activo subyacente.

La primera línea de código utiliza la función "np.maximum" para calcular el pago de la opción de compra. La función "np.maximum" toma dos argumentos: los precios simulados del activo subyacente (St) y el precio de ejercicio de la opción de compra (K). La función devuelve el valor máximo entre la diferencia entre los precios simulados del activo subyacente y el precio de ejercicio (St - K) y cero. Esto asegura que el pago de la opción de compra siempre sea cero si el precio del activo subyacente en el futuro es menor que el precio de ejercicio, y de lo contrario, sea igual a la diferencia entre el precio simulado del activo subyacente y el precio de ejercicio.

La segunda línea de código utiliza la misma función "np.maximum" para calcular el pago de la opción de venta. En este caso, la función devuelve el valor máximo entre la diferencia entre el precio de ejercicio y los precios simulados del activo subyacente (K - St) y cero. Esto asegura que el pago de la opción de venta siempre sea cero si el precio del activo subyacente en el futuro es mayor que el precio de ejercicio, y de lo contrario, sea igual a la diferencia entre el precio de ejercicio y el precio simulado del activo subyacente.

In [52]:

call_payoff = np.maximum(media_trayectorias - K, 0)  # Pago de la opción de compra (call)
put_payoff = np.maximum(K - media_trayectorias, 0)  # Pago de la opción de venta (put)m

In [53]:
call_payoff.mean()

9.61546138148593

In [54]:
put_payoff.mean()

44.816944762504114

**Cálculo del valor presente de las opciones mediante la obtención del promedio de los pagos de las opciones y su descontado a su valor presente**

Estas líneas de código se utilizan para calcular el valor presente de una opción de compra (call) y una opción de venta (put), respectivamente, a partir de los pagos de opciones calculados previamente y teniendo en cuenta la tasa de interés libre de riesgo.

La primera línea de código utiliza la fórmula de valor presente para calcular el valor presente de la opción de compra. La fórmula tiene en cuenta el pago de la opción de compra calculado previamente (call_payoff), la tasa de interés libre de riesgo (r) y el tiempo hasta el vencimiento de la opción (T). La función "np.mean" se utiliza para calcular la media aritmética de los pagos de opciones. Luego, se multiplica el valor promedio de los pagos de opciones por el factor de descuento del valor presente (np.exp(-r * T)) para obtener el valor presente de la opción de compra.

La segunda línea de código utiliza la misma fórmula de valor presente para calcular el valor presente de la opción de venta. En este caso, se utiliza el pago de la opción de venta calculado previamente (put_payoff) en lugar del pago de la opción de compra. Se utiliza la misma función "np.mean" para calcular la media aritmética de los pagos de opciones, y luego se multiplica por el factor de descuento del valor presente (np.exp(-r * T)) para obtener el valor presente de la opción de venta.

In [55]:
T = 365/365
call_value = np.exp(-r * T) * np.mean(call_payoff)  # Valor presente de la opción de compra (call)
put_value = np.exp(-r * T) * np.mean(put_payoff)  # Valor presente de la opción de venta (put)

**Impresión de los resultados de la valuación de las opciones**

In [56]:
print(f"Valor de la opción call: {call_value:.4f}")
print(f"Valor de la opción put: {put_value:.2f}")

Valor de la opción call: 9.1465
Valor de la opción put: 42.63
